In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [1]:
import nltk
from nltk.corpus import opinion_lexicon
from nltk.corpus import wordnet

# Завантажуємо необхідні ресурси NLTK
nltk.download('opinion_lexicon')
nltk.download('wordnet')
nltk.download('omw-1.4') # Open Multilingual Wordnet

# Завантажуємо списки позитивних та негативних слів
positive_words = set(opinion_lexicon.positive())
negative_words = set(opinion_lexicon.negative())

print(f"Кількість позитивних слів: {len(positive_words)}")
print(f"Кількість негативних слів: {len(negative_words)}")

# (Опціонально) Функція для розширення списку слів синонімами з WordNet
def expand_with_synonyms(word_set, lang='eng'):
    synonyms = set()
    for word in word_set:
        for syn in wordnet.synsets(word, lang=lang):
            for lemma in syn.lemmas(lang=lang):
                synonyms.add(lemma.name().lower().replace('_', ' '))
    return word_set.union(synonyms)

# Розширюємо наші лексикони
# Увага: цей процес може додати шум, тому використовувати обережно.
# positive_words_expanded = expand_with_synonyms(positive_words)
# negative_words_expanded = expand_with_synonyms(negative_words)

# print(f"Кількість розширених позитивних слів: {len(positive_words_expanded)}")
# print(f"Кількість розширених негативних слів: {len(negative_words_expanded)}")

[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Unzipping corpora/opinion_lexicon.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Кількість позитивних слів: 2006
Кількість негативних слів: 4783


In [2]:
# Адаптація функції count_tweets
def build_freqs(positive_words, negative_words):
    freqs = {}
    # Мітка 1.0 для позитивних, 0.0 для негативних
    for word in positive_words:
        freqs[(word, 1.0)] = 1
    for word in negative_words:
        freqs[(word, 0.0)] = 1
    return freqs

freqs = build_freqs(positive_words, negative_words)

In [3]:
import numpy as np

def train_naive_bayes_from_lexicon(freqs, positive_words, negative_words):
    loglikelihood = {}

    # Встановлюємо logprior в 0, оскільки припускаємо P(pos) = P(neg)
    # logprior = log(P(pos)/P(neg)) = log(1) = 0
    logprior = 0.0

    # Створюємо словник унікальних слів
    vocab = set([key[0] for key in freqs.keys()])
    V = len(vocab)

    # N_pos та N_neg - загальна кількість слів у кожному класі
    # Оскільки ми присвоїли частоту 1 кожному слову, це просто розмір списків
    N_pos = len(positive_words)
    N_neg = len(negative_words)

    # Розрахунок loglikelihood для кожного слова
    for word in vocab:
        # Частота слова у позитивному та негативному класах
        freq_pos = freqs.get((word, 1.0), 0)
        freq_neg = freqs.get((word, 0.0), 0)

        # Ймовірність слова для кожного класу зі згладжуванням Лапласа
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        loglikelihood[word] = np.log(p_w_pos / p_w_neg)

    return logprior, loglikelihood

logprior, loglikelihood = train_naive_bayes_from_lexicon(freqs, positive_words, negative_words)

In [8]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import string

def process_text(text):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    text = re.sub(r'#', '', text) # Видалення хештегів
    tokens = word_tokenize(text)

    clean_tokens = []
    for word in tokens:
        word = word.lower()
        if (word not in stopwords_english and
            word not in string.punctuation):
            stem_word = stemmer.stem(word)
            clean_tokens.append(stem_word)
    return clean_tokens

def naive_bayes_predict(text, logprior, loglikelihood):
    processed_text = process_text(text)
    p = logprior

    for word in processed_text:
        if word in loglikelihood:
            p += loglikelihood[word]

    return p

# Тестування на прикладі
my_sentence = "This movie was not good, it was actually very bad and boring."
prediction = naive_bayes_predict(my_sentence, logprior, loglikelihood)
print(f"Показник тональності: {prediction}")
print(f"Прогнозована тональність: {'Позитивна' if prediction > 0 else 'Негативна'}")

Показник тональності: 0.13031348927634978
Прогнозована тональність: Позитивна


In [9]:
# Створення власного тестового набору
test_x = [
    'I am happy because I am learning.',
    'This is a wonderful experience.',
    'I feel sad and tired.',
    'This was a terrible movie.',
    'The service was acceptable.',
    'I am not happy with the result.',
    'The book is amazing and fantastic.'
]
# Мітки: 1 для позитивних, 0 для негативних
test_y = np.array([1, 1, 0, 0, 1, 0, 1])

def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    y_hats = []
    for text in test_x:
        if naive_bayes_predict(text, logprior, loglikelihood) > 0:
            y_hat_i = 1
        else:
            y_hat_i = 0
        y_hats.append(y_hat_i)

    y_hats = np.array(y_hats)
    accuracy = np.mean(y_hats == test_y)
    return accuracy

accuracy = test_naive_bayes(test_x, test_y, logprior, loglikelihood)
print(f"Точність класифікатора на власному тестовому наборі: {accuracy:.4f}")

Точність класифікатора на власному тестовому наборі: 0.5714


In [10]:
sorted_loglikelihood = sorted(loglikelihood.items(), key=lambda x: x[1], reverse=True)

print("Топ-10 найбільш позитивних слів:")
for word, value in sorted_loglikelihood[:10]:
    print(f"{word}: {value:.4f}")

print("\nТоп-10 найбільш негативних слів:")
for word, value in sorted_loglikelihood[-10:]:
    print(f"{word}: {value:.4f}")

Топ-10 найбільш позитивних слів:
upliftment: 0.9676
captivating: 0.9676
endorsement: 0.9676
rock-star: 0.9676
exquisite: 0.9676
enjoyably: 0.9676
distinction: 0.9676
complimentary: 0.9676
rightness: 0.9676
cheerful: 0.9676

Топ-10 найбільш негативних слів:
blatantly: -0.4187
alarmingly: -0.4187
misinterpret: -0.4187
scratch: -0.4187
vague: -0.4187
culpable: -0.4187
anomalous: -0.4187
sinking: -0.4187
diabolical: -0.4187
intimidate: -0.4187


In [11]:
print('Аналіз помилок:')
print('Справжня | Прогноз  | Текст')
print('------------------------------------')
for x, y in zip(test_x, test_y):
    prediction = naive_bayes_predict(x, logprior, loglikelihood)
    y_hat = 1 if prediction > 0 else 0
    if y != y_hat:
        print(f"   {y}    |    {y_hat}     | {x}")

Аналіз помилок:
Справжня | Прогноз  | Текст
------------------------------------
   1    |    0     | I am happy because I am learning.
   1    |    0     | The service was acceptable.
   1    |    0     | The book is amazing and fantastic.


In [12]:
import json

data_to_save = {
    'logprior': logprior,
    'loglikelihood': loglikelihood
}

with open('naive_bayes_model.json', 'w') as f:
    json.dump(data_to_save, f)